In [7]:
#environment setting
import pandas as pd
import sqlite3
import re
# Library Import and Environment Setting needed for Logistic Regression

import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

plt.rc("font", size=16)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import os
import os.path
import glob

import json

from collections import defaultdict

! pip install profanity-check
from profanity_check import predict, predict_prob
# https://github.com/vzhou842/profanity-check

In [5]:
os.chdir('/Users/alexpapaioannou/Documents/GitHub/TFA_Project/')

raw_filenames = []
artists = []

# Get .txt files names
for file_name in os.listdir('Lyrics'):
    if file_name.endswith('.txt'):
        raw_filenames.append(file_name)
        artists.append(file_name.split("~")[2].split(".txt")[0])

def raw_filenames_list(str)->list:
    raw_filenames_list_ = []
    for filename in os.listdir('Lyrics'):
        if filename.endswith(str):
            raw_filenames_list_.append(filename)
    return raw_filenames_list_

print(raw_filenames_list('.txt'))
print(len(raw_filenames_list('.txt')))

def artists_list(str)->list:
    artists_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            artists_list_.append(filename.split("~")[2].split(".txt")[0])
    return artists_list_

print(artists_list('.txt'))
print(len(artists_list('.txt')))
# print(set(artists_list('.txt')))
print(len(set(artists_list('.txt'))))

def songs_list(str)->list:
    songs_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            songs_list_.append(filename.split("~")[1].split(".txt")[0])
    return songs_list_

print(songs_list('.txt'))
print(len(songs_list('.txt')))

def song_ordering_numbers_list(str)->list:
    song_ordering_numbers_list_ = []
    for filename in raw_filenames:
        if filename.endswith(str):
            song_ordering_numbers_list_.append(filename.split("~")[0].split(".txt")[0])
    return song_ordering_numbers_list_

print(song_ordering_numbers_list('.txt'))
print(len(song_ordering_numbers_list('.txt')))

import glob

def artist_s_songs_list(str)->list:
    artist_s_songs_list_ = []
    for raw_filename in os.listdir('Lyrics'):
        if raw_filename.endswith(str+'.txt'):
#           print("For the artist", str, "the file(s)", raw_filenames, "has the keyword")
            artist_s_songs_list_.append(raw_filename)
#           print(artist_s_songs_list_)
    return artist_s_songs_list_

for artist in set(artists):
    print(artist_s_songs_list(artist))

['424~We Care~Reel Big Fish.txt', '122~Hallelujah I Just Love Her So~Richard "Cookie" Thomas.txt', '117~Clink, Clink, Another Drink~Spike Jones & His City Slickers.txt', "774~Ain't Misbehavin'~Anita O'Day.txt", '498~A Fool In Love~Tina Turner.txt', "389~Quinceañera~Los Yonic's.txt", '422~My Bonnie~The Bristols.txt', '37~[Silence]~Buy This Song.txt', "289~Climb Ev'ry Mountain~Patricia Neway.txt", '847~Love Never Loved Me~Geri Halliwell.txt', '247~Blueberry Hill~John Fahey.txt', '500~Lose It~Lil Wayne.txt', '689~Reynard the Fox~Fairport Convention.txt', '896~Sad and Lonely~Secret Machines.txt', '766~You Really Got Me~The Kinks.txt', '964~Te Vas Angel Mio~Cornelio Reyna.txt', "116~It's You or No One~Art Blakey & the Jazz Messengers.txt", '460~Cut Her Off~2 Chainz.txt', '581~A Message to You Rudy~The Specials.txt', '724~Goodnight Sweetheart~James Booker.txt', '26~Alles Klar auf der Andrea Doria~Udo Lindenberg.txt', "934~You Don't Mess Around with Jim~Jim Croce.txt", '573~Copacabana~Matteo 

In [14]:
# os.mkdir('/Users/alexpapaioannou/Documents/GitHub/TFA_Project/Lyrics/Cleaned_Songs') #uncomment if the directory not present
for artist in set(artists):
    for song in artist_s_songs_list(artist):
        f = open('Lyrics/' + song, 'rb')
        all_words = ''
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            all_words += this_sentence
        #remove identifiers like chorus, verse, etc
        all_words = re.sub(r'[\(\[].,;*?[\)\]]', '', all_words)
        #remove empty lines
        all_words = os.linesep.join([s for s in all_words.splitlines() if s])
        f.close()
        directory = "Lyrics/Cleaned_Songs"
        f = open(os.path.join(directory, 'cleaned_' + song ), "wb")
        f.write(all_words.encode('utf-8'))
        f.close()

In [15]:
plt.rcParams['figure.figsize'] = (126,190)

df = pd.DataFrame(columns=('artist', 'words'))
i=0
            
def artist_s_cleaned_songs_list(str)->list:
    artist_s_cleaned_songs_list_ = []
    for cleaned_raw_filename in os.listdir("Lyrics/Cleaned_Songs/"):
        if cleaned_raw_filename.endswith(str + '.txt') is True:
            if cleaned_raw_filename.endswith(str + '.txt') not in artist_s_cleaned_songs_list_:
                artist_s_cleaned_songs_list_.append(cleaned_raw_filename)
    return artist_s_cleaned_songs_list_


# for artist in set(artists):
#     print(artist, '\n', artist_s_cleaned_songs_list(artist))

for artist in set(artists):
    num_words = 0
    all_words = ''
    for song in artist_s_cleaned_songs_list(artist):
        f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            num_words_this = len(this_sentence.split(" "))
            num_words += num_words_this


    df.loc[i] = (artist, num_words)
    i+=1

df.plot.bar(x='artist', y='words', title='Number of Words for each Artist');
plt.savefig('Number_of_Words_for_each_Artist_result.pdf')
plt.show()

In [17]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
sid = SentimentIntensityAnalyzer()
i=0

os.chdir('/Users/alexpapaioannou/Documents/GitHub/TFA_Project/')

num_neutral = 0
num_negative = 0
num_positive = 0
    
for artist in set(artists):
    
    num_neutral = 0
    num_negative = 0
    num_positive = 0

    for song in artist_s_cleaned_songs_list(artist):
        f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
        for sentence in f.readlines():
            this_sentence = sentence.decode('utf-8')
            comp = sid.polarity_scores(this_sentence)
            comp = comp['compound']
            if comp >= 0.5:
                num_positive += 1
            elif comp > -0.5 and comp < 0.5:
                num_neutral += 1
            else:
                num_negative += 1
                
    num_total = num_negative + num_neutral + num_positive
    if num_total != 0:
        percent_negative = (num_negative/float(num_total))*100
        percent_neutral = (num_neutral/float(num_total))*100
        percent_positive = (num_positive/float(num_total))*100
        df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
        i+=1
    else:
        percent_negative = 0
        percent_neutral = 0
        percent_positive = 0
        df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
        i+=1

df.plot.bar(x='artist', stacked=True)
# plt.tick_params(axis = 'x', direction='out', length=26, width=49)
plt.savefig('Vocabulary_Richness_for_each_Artist_result.pdf')
plt.show()